<a href="https://colab.research.google.com/github/reznimat/ppzd_sp/blob/main/semestralni_prace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Načtení knihoven**

In [65]:
import csv
import pandas as pd
import math as mt

**Načtení datových souborů**

In [66]:
# POZOR, pouze vzorek dat
url_sales = 'https://raw.githubusercontent.com/reznimat/ppzd_sp/main/SAMPLE_Online_Sales.csv'
#url_sales = 'https://raw.githubusercontent.com/reznimat/ppzd_sp/main/Online_Sales.csv'
df_sales = pd.read_csv(url_sales)

url_coupons = 'https://raw.githubusercontent.com/reznimat/ppzd_sp/main/Discount_Coupon.csv'
df_coupons = pd.read_csv(url_coupons,sep=";")

print('**Sales**')
print(df_sales.count())
print('\n------\n')
print('**Coupons**')
print(df_coupons.count())


**Sales**
CustomerID             30
Transaction_ID         30
Transaction_Date       30
Product_SKU            30
Product_Description    30
Product_Category       30
Quantity               30
Avg_Price              30
Delivery_Charges       30
Coupon_Status          30
dtype: int64

------

**Coupons**
Month               204
Product_Category    204
Coupon_Code         204
Discount_pct        204
dtype: int64


**Obohacení df_sales o atribut Month (Date Abrv)**

In [67]:
# Obohacení df_sales o Month pole
for i, row in df_sales.iterrows():
  Month = pd.to_datetime(row.Transaction_Date).strftime("%b")
  df_sales["Month"] = Month

print('**Sales**')
print(df_sales.count())

**Sales**
CustomerID             30
Transaction_ID         30
Transaction_Date       30
Product_SKU            30
Product_Description    30
Product_Category       30
Quantity               30
Avg_Price              30
Delivery_Charges       30
Coupon_Status          30
Month                  30
dtype: int64


**Spojení sales a coupons tabulek přes Product_Category a Month**

In [68]:
#Merge Sales a Coupons na základě Product Category a Month
df_sales_coupons = pd.merge(df_sales,df_coupons, on=["Product_Category","Month"])

print(df_sales_coupons.count())

CustomerID             30
Transaction_ID         30
Transaction_Date       30
Product_SKU            30
Product_Description    30
Product_Category       30
Quantity               30
Avg_Price              30
Delivery_Charges       30
Coupon_Status          30
Month                  30
Coupon_Code            30
Discount_pct           30
dtype: int64


**Počet a výpis jedinečných produktových kategorií**

In [78]:
# Počet a výpis distinktních produktových kategorií
uniqPC = set()
for i, row in df_sales.iterrows():
  uniqPC.add(row["Product_Category"])

print("Produktových kategorií: "+ str(len(uniqPC)) )  # vypíše počet produktových kategorií
print(uniqPC)  # vypíše seznam produktových kategorií

Produktových kategorií: 6
{'Bags', 'Nest-USA', 'Apparel', 'Office', 'Lifestyle', 'Drinkware'}


**Suma tržeb pro kategorii Apparel**

In [70]:
# Hint: InvoiceValue = (( QuantityAvg_price)(1-Dicount_pct)*(1+GST))+Delivery_Charges
# nejsou započítány náklady na dopravu a daně

ApparelSales = 0
AvgOrderPrice = 0

for i, row in df_sales_coupons.iterrows():
  if row.Product_Category == 'Apparel':
    if row.Coupon_Status =='Not Used':
      ApparelSales += (row.Quantity*row.Avg_Price) # Bez slevy
    else:
      ApparelSales += (row.Quantity*row.Avg_Price)*(1-(row.Discount_pct/100)) # Se slevou

print(ApparelSales)

191.77100000000002


In [85]:
uniqMnth = set()
for i, row in df_sales.iterrows():
  uniqMnth.add(row["Month"])

print(uniqMnth)

{'Jan'}


**Tržby po kategoriích a měsících**

In [91]:

for kategorie in uniqPC:
  for mesic in uniqMnth:
    CategorySales = 0
    for i, row in df_sales_coupons.iterrows():
      if row.Product_Category == kategorie and row.Month == mesic:
        if row.Coupon_Status =='Not Used':
          CategorySales += (row.Quantity*row.Avg_Price) # Bez slevy
        else:
          CategorySales += (row.Quantity*row.Avg_Price)*(1-(row.Discount_pct/100)) # Se slevou

    print(kategorie, mesic, CategorySales)

Bags Jan 84.375
Nest-USA Jan 819.846
Apparel Jan 191.77100000000002
Office Jan 181.77900000000002
Lifestyle Jan 8.343
Drinkware Jan 441.86800000000005
